In [77]:
from langchain.llms import OpenAI
from langchain_groq import ChatGroq

In [78]:
import os
import streamlit as st
os.environ["GROQ_API_KEY"] = st.secrets["GROQ_API_KEY"]

In [79]:
llm = ChatGroq(model="llama3-8b-8192", temperature=0.7)

In [80]:
text = "What is the capital of Bharat?"
print(llm.predict(text))

A nice question!

The capital of India, which is also known as Bharat, is New Delhi.


In [81]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = st.secrets["HUGGINGFACEHUB_API_TOKEN"]

In [82]:
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0.7})

In [83]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-large")
output = pipe("What is the capital of Russia?")[0]["generated_text"]
print(output)


Device set to use cpu


moscow


In [108]:
output = pipe("How New Delhi relates to India?")[0]["generated_text"]
print(output)

the capital city of india


In [85]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],
template="Tell me the capital of this {country}")

prompt_template.format(country="India")

'Tell me the capital of this India'

In [86]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("India"))

I think there may be a slight misunderstanding! India is a country, not a state or a province, so it doesn't have multiple capitals. The capital of India is a single city, which is:

New Delhi ( Hindi: नई दिल्ली)

Let me know if you have any further questions!


In [87]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template)

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

In [88]:
famous_chain=LLMChain(llm=llm,prompt=famous_template)

In [89]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
print(chain.run("India"))

The vibrant and bustling city of New Delhi! There are countless amazing places to visit in the capital of India, rich in history, culture, and architecture. Here are some must-visit attractions to add to your itinerary:

1. **Red Fort (Lal Qila)**: A majestic Mughal fort built in the 17th century, now a UNESCO World Heritage Site. It's a symbol of India's rich history and architecture.
2. **Qutub Minar**: The tallest minaret in India, built in 1192, is an iconic landmark and one of the Seven Wonders of the Medieval World.
3. **India Gate**: A war memorial built to commemorate the Indian soldiers who died during World War I. It's a popular spot for sunset views and picnics.
4. **Humayun's Tomb**: A beautiful example of Mughal architecture, this tomb is a precursor to the Taj Mahal and is also a UNESCO World Heritage Site.
5. **Akshardham Temple**: A stunning Hindu temple complex with beautiful architecture, water features, and cultural exhibits.
6. **Rashtrapati Bhavan**: The official r

In [90]:
capital_template=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of the {country}")

capital_chain=LLMChain(llm=llm,prompt=capital_template,output_key="capital")

In [91]:
famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}")

famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [92]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])

In [93]:
chain({'country':"India"})

{'country': 'India',
 'capital': 'The capital of India is New Delhi.',
 'places': "New Delhi, the capital city of India, is a treasure trove of history, culture, and architecture. Here are some amazing places to visit in New Delhi:\n\n1. **Red Fort (Lal Qila)**: A magnificent Mughal palace built in 1648, the Red Fort is a must-visit attraction in Delhi. It's a UNESCO World Heritage Site and a symbol of India's rich history.\n2. **Qutub Minar**: The tallest minaret in India, Qutub Minar is a stunning example of Islamic architecture. It's a must-visit attraction in Delhi and a popular spot for photography.\n3. **India Gate**: A war memorial built to honor the Indian soldiers who died during World War I, India Gate is a beautiful structure surrounded by lush green gardens.\n4. **Akshardham Temple**: A modern Hindu temple built in 2005, Akshardham is a beautiful example of Indian architecture and culture. The temple complex features stunning gardens, water features, and a massive statue of

In [94]:
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [95]:
chatllm = ChatGroq(model="llama3-8b-8192", temperature=0.7) 

In [96]:
chatllm([SystemMessage(content="You are a helpful assistant."),
         HumanMessage(content="What is the capital of Bharat?")])

AIMessage(content='The capital of Bharat is New Delhi.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 29, 'total_tokens': 39, 'completion_time': 0.008333333, 'prompt_time': 0.008761469, 'queue_time': 0.24538021899999998, 'total_time': 0.017094802}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run--08aba60c-2e53-4858-95c9-39f541a8744b-0', usage_metadata={'input_tokens': 29, 'output_tokens': 10, 'total_tokens': 39})

In [97]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import BaseOutputParser

In [98]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [99]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)])

In [100]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [101]:
chain.invoke({"text":"intelligent"})

['Brilliant', ' Smart', ' Intelligent', ' Clever', ' Astute']